In [1]:
from scripts.imports import os, glob, pdb, np, h5py, pd, xr, gpd, Proj, Transformer, CRS, \
                        plt, cmap, Model, Data, ODR, datetime, rasterio, show, \
                        ccrs, cfeature
from scripts.classes_fixed import *
from scripts.pvpg_fixed import *
from scripts.track_pairs import *

/home/s1803229/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


# Idea: Manual $\rho_v/\rho_g$

Right now I have relative height and classification from ATL08. This drops about 2/3 of the photons, and I can't get latitude, longitude, and actual height without linking to ATL03.

BUT

I know what the segment ID is. That means I know its location within 20m. We can still compute Eg and Ev within (and aggregated across) segments using just ATL08. We can use Katie's formula to compute the average noise to take away from the canopy, and then we have our corrected Ev. We then plot this manual computation and compare to what we've got from the terrible stuff we've seen so far.

As a side note, if I can find a way to link the classed_pc_indx to where the photon is in the ATL03 list of photons, then we should be able to fix the classification problem. This is even better, because then I can fix the whole problem that for some reason 'gt1l' for track [N-4] is an edge case that isn't handled by either PhoREAL or Matt's classes. So that's brilliant. I can see where I need to fix it, but I don't know how to fix it. If there weren't enough classifications for the photon, the last photons are simply classified as unclassified. Now, if there are more photons than classifications, which photons do I miss out? The last ones? Where did these extra classifications come from. They could be from the middle, and then suddenly half the track will be messed up. I'll try this later.

### Dataframe with classified photons and their segment IDs in the ATL08 file - Can we use this for manual $\rho_v/\rho_g$?

In [2]:
%matplotlib ipympl

dirpath = '../data/rovaniemi_w'
all_ATL03, all_ATL08 = track_pairs(dirpath)

N = len(all_ATL03)

In [3]:
A = h5py.File(all_ATL03[N-1], 'r')
B = h5py.File(all_ATL08[N-1], 'r')

gt = 'gt1r'
h = np.array(B[gt + '/signal_photons/ph_h'][:])
c = np.array(B[gt + '/signal_photons/classed_pc_flag'][:])
s = np.array(B[gt + '/signal_photons/ph_segment_id'][:])
df = pd.DataFrame()
df['h'] = h
df['c'] = c
df['seg_id'] = s
display(df[df.c == 1])
display(df[df.c == 2])
display(df[df.c == 3])

,h,c,seg_id
1,0.010651,1,631298
2,0.244980,1,631298
6,0.356354,1,631298
11,-0.376923,1,631299
14,-0.132126,1,631299
...,...,...,...
7270,0.349678,1,631900
7272,0.055351,1,631901
7274,0.376717,1,631901
7278,0.344460,1,631902


,h,c,seg_id
0,0.643341,2,631298
4,0.534668,2,631298
7,0.825912,2,631299
8,4.961090,2,631299
22,3.238388,2,631299
...,...,...,...
6611,1.058655,2,631847
6612,0.767242,2,631847
6613,7.101685,2,631847
6614,0.860275,2,631847


,h,c,seg_id
3,10.956711,3,631298
12,10.190491,3,631299
49,11.767120,3,631300
58,10.829391,3,631300
59,13.147858,3,631300
...,...,...,...
6585,9.390915,3,631845
6586,9.493744,3,631845
6587,10.649780,3,631845
6597,8.882050,3,631846


### Construction of the manual $\rho_v/\rho_g$ function

In [ ]:
def plot(df, ax):
    for c in np.unique(df.classification):
        mask = df.classification==c
        ax.scatter(df[mask].lat_ph,
                   df[mask].h_ph,
                   color=class_dict[c]['color'],
                   label=class_dict[c]['name'],
                   s = 5)
    ax.set_xlabel('Latitude (°)')
    ax.set_ylabel('Elevation (m)')
    ax.legend(loc='best')

def pvpg_manual(atl03path, atl08path):
    
    i = 0
    
    tracks = ['gt1r', 'gt1l', 'gt2r', 'gt2l', 'gt3r', 'gt3l']
    A = h5py.File(atl03path, 'r')
    B = h5py.File(atl08path, 'r')
    
    fig, axes = plt.subplots(6, 2, figsize=(8, 30))
    ax = axes.flatten()
    
    # Extracting date and time from the filename
    title_date = parse_filename_datetime(atl03path)

    # Set the figure title
    fig.suptitle(title_date, fontsize=16)

    for gt in tracks:
        
        # 1) First we need quality control:
        
        # 1a) If there's no data in the track, it needs to be thrown out
            
        if not 'heights' in A[gt].keys():
            i += 2
            continue
            
            # 1b) We need to link the signal photons to their classifications
            # and plot the track
            
        def classifyphotons(A, B):
            
            h = np.array(B[gt + '/signal_photons/ph_h'][:])
            c = np.array(B[gt + '/signal_photons/classed_pc_flag'])
            seg = np.array(B[gt + '/signal_photons/ph_segment_id'][:])
            df = pd.DataFrame()
            df['h'] = h
            df['c'] = c
            df['seg_id'] = seg
            return df
            # 1c) I can see based on an eye test if a track is useless, so I can write an algorithm that can see
            # if a track is useless. in these cases, I would like to plot the track, but not pv/pg
            
            # 1d) I only want to do my computations on the parts of the track that are useful. I can identify
            # these places by an eye test, so I can find them with an algorithm
            
        # 2) We need to plot the track on the left plot
        
            # 2a) We need to link the signal photons to their classifications
            #     We could also just use the method we already know
            
        # 3) Now that we have the good parts of the track, we need to compute Ev and Eg for various segments
        
            # 3a) We need to make sure our calculations are done on continuous segments
            
            # 3b) For the canopy, we need to make use of ATL08 to do noise correction
            
            # 3c) Make sure there's no funny business like the E+38 stuff

        # 4) Now we do the orthogonal regression and plot the result

        def linear_model(params, x):
            return params[0]*x + params[1]

        linear = Model(linear_model)
        data = Data(atl08.df.Eg,atl08.df.Ev)
        odr = ODR(data, linear, beta0 = [1.0,1.0])
        result = odr.run()
        slope, intercept = result.beta

        ax[i+1].set_title(f"{gt} 100m Photon Rates")
        ax[i+1].scatter(atl08.df.Eg, atl08.df.Ev, s=10)
        ax[i+1].plot([0,-intercept/slope],[intercept,0])
        ax[i+1].set_xlabel('Eg (returns/shot)')
        ax[i+1].set_ylabel('Ev (returns/shot)')
        ax[i+1].annotate(r'$\rho_v/\rho_g \approx {:.2f}$'.format(-slope),
                       xy=(.95,.95),
                       xycoords='axes fraction',
                       ha='right',
                       va='top',
                       bbox=dict(boxstyle="round,pad=0.3",
                                 edgecolor="black",
                                 facecolor="white"))
        i += 2
    
    plt.tight_layout(rect=[0, 0, 1, 0.97])  # Adjust the layout to make room for the suptitle
    plt.show()
    return